In [14]:
import csv
import pandas as pd

# cleans up list elements by removing colon suffix
def cleanTrail(listID):
    for i in range(len(listID)):
        if ":" in listID[i]:
            listID[i] = listID[i][:-3]
            
#pulls transaciton set out of map name            
def sliceTSet(mapName):
    if 'ansix12' in mapName:
        tSet = mapName[13:16]
    else:
        tSet = (mapName[13:19]).upper()
    return tSet   

#pulls version out of map name
def sliceVersion(mapName):
    if 'ansix12' in mapName:
        version = '00' + mapName[17:21]
    else:
        version = 'D ' + mapName[19:22]
    return version

#Dictionary that will be converted to dataframe
d = {'Sender':[], 'Transaction Set':[], 'Version':[], 'EDICode':[], 'TSIM Map':[]}

with open('TSIMOE_Inbound.csv') as f:
    #gets rid of headers
    f.readline()
    csv_reader = csv.reader(f,delimiter = ',')
    
    #iterate through rows in CSV file
    for line_list in csv_reader:
        #create lists using delimited values from SENDER_EXT and RECEIVER_EXT
        sender_list = [x.strip() for x in line_list[4].split('|')]
        receiver_list = [x.strip() for x in line_list[5].split('|')]
        
        cleanTrail(sender_list)
        cleanTrail(receiver_list) 
        
        #loop thorugh senders first to ensure all possible combinations are made
        for sender in sender_list:
            # in the case that there is no TSIM map available 
            if not line_list[10]:
                continue
            for receiver in receiver_list:
                d['Sender'].append(line_list[0])
                d['Transaction Set'].append(sliceTSet(line_list[10]))
                d['Version'].append(sliceVersion(line_list[10]))
                d['EDICode'].append(sender + receiver)
                d['TSIM Map'].append(line_list[10])
                
df = pd.DataFrame(data=d)
#print(df)

#write DataFrame to a file
writer = pd.ExcelWriter('gotEm6.xlsx', engine='xlsxwriter')
df.to_excel(writer)
writer.save()      